In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [2]:
from conllu import parse
import pandas as pd
import os

# Define the language code, used in the file names
lang_code = "FI"

# Main path
main_path = "/home/tajak/Parlamint-translation"

# Check whether the path to the folder with conllu files is ok
path = "{}/Source-data/ParlaMint-{}.conllu/ParlaMint-{}.conllu".format(main_path, lang_code, lang_code)

# Define other paths
extracted_dataframe_path = "{}/results/{}/ParlaMint-{}-extracted-source-data.csv".format(main_path, lang_code, lang_code)

We need to translate the following corpora into English:
- Belgian (BE) - in Dutch and French, separate CONLL-us (!!). French: ["fr", "itc","roa"], Dutch: ["nl", "gem", "gmw"]
- Bulgarian (BG): ["bg", "sla", "zls"]
- Croatian (HR) -> "zls"
- Czech (CZ) -> "cs"
- Danish (DK): ["da", "gmq", "gem"]
- Dutch (NL): ["nl", "gem", "gmw"]
- French (FR): ["fr", "itc","roa"]
- Hungarian (HU): ["hu", "fiu", "urj"]
- Icelandic (IS): ["is","gmq", "gem"]
- Italian (IT): ["it", "roa", "itc"]
- Latvian (LV): ["lv", "bat"]
- Lithuanian (LT): only "bat"
- Polish (PL): ["pl", "sla", "zlw"]
- Slovenian (SI) - We will use "Slavic MT" based on the results of the manual analysis --> "sla"
- Spanish? (ES): ["es", "roa", "itc"]
- Turkish (TR): ["tr", "trk" ]
- Austrian (AT): ["de", "gem", "gmw"]
- Basque (ES-PV): ["eu", "mul"]
- Bosnian (BA): ["sla", "zls"]
- Catalan (ES-CT): ["ca", "roa", "itc"]
- Estonian (EE): ["et", "urj", "fiu"]
- Finnish (FI): ["fi", "urj", "fiu"]
- Galician (ES-GA): ["gl", "roa", "itc"]
- Greek (GR):  ["el", "grk"]
- Norwegian (NO): ["gem", "gmq"]
- Portuguese (PT): ["pt", "roa", "itc"]
- Romanian (RO): ["roa", "itc"]
- Serbian (RS): ["sla", "zls"]
- Swedish (SE): ["sv", "gmq", "gem"]
- Ukrainian (UA): ["uk", "sla", "zle"]
- Russian (present in UA): ["ru", "sla", "zle"]

Languages with only one option: Lithuanian ("LT": "bat")

Explanation of language codes:
- sla = Slavic
- zls = South Slavic
- zlw = West Slavic
- zle = East Slavic
- gmq = North Germanic
- gem = Germanic
- gmw = West Germanic
- roa = Romance
- itc = Italic
- bat = Baltic
- trk = Turkic
- urj = Uralic
- fiu = Finno-Ugrian

In [3]:
from easynmt import EasyNMT

# Define the model
model = EasyNMT('opus-mt')

print("\n\nAll languages with target_lang=de. I.e., we can translate from these languages to English (en).")
print(model.get_languages(source_lang='en'))



/home/tajak/Parlamint-translation/parlamint_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-26 10:33:10.643231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 10:33:11.344969: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-26 10:33:11.345050: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'l



All languages with target_lang=de. I.e., we can translate from these languages to English (en).
['aav', 'af', 'alv', 'ar', 'az', 'bat', 'bcl', 'bem', 'ber', 'bg', 'bi', 'bnt', 'bzs', 'ca', 'ceb', 'cel', 'chk', 'cpf', 'crs', 'cs', 'cus', 'cy', 'da', 'de', 'dra', 'ee', 'efi', 'el', 'eo', 'es', 'et', 'eu', 'euq', 'fi', 'fj', 'fr', 'ga', 'gaa', 'gil', 'gl', 'grk', 'guw', 'gv', 'ha', 'he', 'hi', 'hil', 'ho', 'ht', 'hu', 'hy', 'id', 'ig', 'ilo', 'is', 'iso', 'it', 'jap', 'kg', 'kj', 'kqn', 'kwn', 'kwy', 'lg', 'ln', 'loz', 'lu', 'lua', 'lue', 'lun', 'luo', 'lus', 'map', 'mfe', 'mg', 'mh', 'mk', 'mkh', 'ml', 'mos', 'mr', 'mt', 'mul', 'ng', 'nic', 'niu', 'nl', 'nso', 'ny', 'nyk', 'om', 'pag', 'pap', 'phi', 'pis', 'pon', 'poz', 'pqe', 'pqw', 'rn', 'rnd', 'ro', 'roa', 'ru', 'run', 'rw', 'sal', 'sg', 'sit', 'sk', 'sm', 'sn', 'sq', 'ss', 'st', 'sv', 'sw', 'swc', 'tdt', 'ti', 'tiv', 'tl', 'tll', 'tn', 'to', 'toi', 'tpi', 'trk', 'ts', 'tut', 'tvl', 'tw', 'ty', 'uk', 'umb', 'ur', 'vi', 'xh', 'zh']


## Create sample (if df has not been extracted yet)

In [9]:
import os

# Define the path to the folder with conllu files is ok:
path = "ParlaMint-{}.conllu".format(lang_code)

# ------------NO CHANGING OF THE CODE NEEDED FROM NOW ONWARDS--------------

# Create a folder with results for this language, e.g. results/CZ
os.mkdir("/home/tajak/Parlamint-translation/results/{}".format(lang_code))

# Create (manually) a "temp" folder inside the results/CZ
os.mkdir("/home/tajak/Parlamint-translation/results/{}/temp".format(lang_code))

# Define final path
extracted_dataframe_path = "{}/results/{}/ParlaMint-{}-extracted-sample.csv".format(main_path, lang_code, lang_code)

# Extract a list with paths to conllu files and a list with their names
parl_list = []
file_name_list = []

for dir1 in os.listdir(path):
    full_path = os.path.join(path, dir1)
    if os.path.isdir(full_path):
        current = os.listdir(full_path)
        # Keep only files with parliamentary sessions:
        for file in current:
            if "ParlaMint-{}_".format(lang_code) in file:
                if ".conllu" in file:
                    final_path = "{}/{}".format(full_path, file)
                    parl_list.append(final_path)
                    file_name_list.append(file)

# See how many files we have:
print("No. of files: {}.".format(len(parl_list)))


# Extract only one file
def conllu_to_df(parl_list, file_name_list, extracted_dataframe_path):
	"""
	Take the conllu files and extract relevant information. Save everything in a DataFrame.

	Args:
	- parl_list: list of documents with their entire paths to be included (see step above).
	- file_name_list: list of names of the files (see step above)
	- extracted_dataframe_path: path to the output file
	"""
	from conllu import parse
	import pandas as pd

	# Create an empty df
	df = pd.DataFrame({"file_path": [""],"file": [""], "sentence_id": [""], "text": [""], "tokenized_text": [""], "proper_nouns": [""]})

	# Check whether there are any problems with parsing the documents
	"""
	
	error_count = 0
	problematic_doc_list = []

	for doc in parl_list:
		try:
			# Open the file
			data = open("{}".format(doc), "r").read()

			sentences = parse(data)
		except:
			error_count += 1
			problematic_doc_list.append(doc)

	print(error_count)
	print(problematic_doc_list)
	"""
	# Parse the data with CONLL-u parser
	for doc in parl_list:
		# Open the file
		data = open("{}".format(doc), "r").read()
		
		sentences = parse(data)

		sentence_id_list = []
		text_list = []
		tokenized_text_list = []
		proper_noun_list = []

		for sentence in sentences:
			# Find sentence ids
			current_sentence_id = sentence.metadata["sent_id"]
			sentence_id_list.append(current_sentence_id)

			# Find text - if texts consists of multiword tokens, these tokens will appear as they are,
			# not separated into subwords
			current_text = sentence.metadata["text"]
			text_list.append(current_text)

			# Create a string out of tokens
			current_token_list = []
			word_dict = {}

			for token in sentence:
				# Find multiword tokens and take their NER
				if type(token["id"]) != int:
					multiword_ner = token["misc"]["NER"]
				
				else:
				# Append to the tokenized text tokens that are not multiword tokens
				# (we append subtokens to the tokenized texts, not multiword tokens)
					current_token_list.append(token["form"])
					

					# Create a list of NE annotations with word indices.
					# I'll substract one from the word index,
					# because indexing in the CONLLU file starts with 1, not 0
					current_index = int(token["id"]) - 1

					# If the word does not have NER annotation,
					# take the annotation from the multiword token
					if token["misc"] is None:
						current_ner = multiword_ner
					else:
						current_ner = token["misc"]["NER"]

					# Add information on the lemma if the NE is personal name
					if "PER" in current_ner:
						word_dict[current_index] = [token["form"], token["lemma"]]

			proper_noun_list.append(word_dict)

			current_string = " ".join(current_token_list)

			tokenized_text_list.append(current_string)

		
		new_df = pd.DataFrame({"sentence_id": sentence_id_list, "text": text_list, "tokenized_text": tokenized_text_list, "proper_nouns": proper_noun_list})

		new_df["file_path"] = doc

		# Get the file name
		file_name = file_name_list[parl_list.index(doc)]
		new_df["file"] = file_name

		# Merge df to the previous df
		df = pd.concat([df, new_df])
	
	# Reset index
	df = df.reset_index(drop=True)

	# Remove the first row
	df = df.drop([0], axis="index")

	# Reset index
	df = df.reset_index(drop=True)

	# Add information on length
	df["length"] = df["text"].str.split().str.len()

	print("Number of words in the corpora: {}".format(df["length"].sum()))

	# Save the dataframe
	df.to_csv("{}".format(extracted_dataframe_path), sep="\t")

	print("Dataframe saved as {}".format(extracted_dataframe_path))
	
	# Show the results
	print(df.describe(include="all").to_markdown())

	print("\n\n\n")

	print(df.head().to_markdown())

	print("\n\n\n")
	
	return df

#Extract information from the conllu files: extract only one file
df = conllu_to_df(parl_list[:1], file_name_list[:1], extracted_dataframe_path)


No. of files: 515.
Number of words in the corpora: 59823
Dataframe saved as /home/tajak/Parlamint-translation/results/HU/ParlaMint-HU-extracted-sample.csv
|        | file_path                                               | file                           | sentence_id                    | text                   | tokenized_text          | proper_nouns   |    length |
|:-------|:--------------------------------------------------------|:-------------------------------|:-------------------------------|:-----------------------|:------------------------|:---------------|----------:|
| count  | 3606                                                    | 3606                           | 3606                           | 3606                   | 3606                    | 3606           | 3606      |
| unique | 1                                                       | 1                              | 3606                           | 2938                   | 2938                    | 407           

## Choose model

In [4]:
def choose_model(lang_code, extracted_dataframe_path):
	"""
	Compare a small sample of translations of all OPUS-MT models that are available
	for the language, to decide which one to use. The function prints out a dataframe with all translations of the sample and saves it as ParlaMint-{lang_code}-sample-model-comparison.csv.

	Args:
	- lang_code: the lang code that is used in the names of the files, it should be the same as for extract_text()
	- extracted_dataframe_path: path to the final output of 1-conllu-to-df.py
	"""
	import pandas as pd
	import regex as re
	from easynmt import EasyNMT
	from IPython.display import display
	
	lang_models_dict = {"BG": ["bg", "sla", "zls"], "HR": ["zls", "sla"], "CZ": ["cs", "sla", "zlw" ], "DK": ["da", "gmq", "gem"], "NL": ["nl", "gem", "gmw"], "FR": ["fr", "itc","roa"], "BE-fr": ["fr", "itc", "roa"], "BE-nl": ["nl", "gem", "gmw"], "HU": ["hu", "fiu", "urj"], "IS": ["is","gmq", "gem"], "IT": ["it", "roa", "itc"], "LV": ["lv","bat"], "LT": ["bat"], "PL": ["pl", "sla", "zlw"], "SI": ["sla"], "ES": ["es", "roa", "itc"], "TR": ["tr", "trk" ], "AT": ["de", "gem", "gmw"], "ES-PV": ["eu", "mul"], "BA": ["sla", "zls"], "ES-CT": ["ca", "roa", "itc"], "EE": ["et", "urj", "fiu"], "FI": ["fi", "urj", "fiu"], "ES-GA": ["gl", "roa", "itc"], "GR": ["grk"], "NO": ["gem", "gmq"], "PT": ["roa", "itc"], "RO":["roa", "itc"], "RS": ["zls", "sla"], "SE": ["sv", "gmq", "gem"], "UA":["uk", "sla", "zle"], "UA_ru":["ru", "sla", "zle"], "BE": ["nl", "gem", "gmw"]}


	# Open the file, created in the previous step
	df = pd.read_csv("{}".format(extracted_dataframe_path), sep="\t", index_col=0, na_filter = False)

	# Define the model
	model = EasyNMT('opus-mt')

	print("Entire corpus has {} sentences and {} words.".format(df["text"].count(), df["length"].sum()))

	if lang_code == "UA":
		# Create a sample, making sure that it is in Ukrainian language and that the file is big enough (has 30 sentences)
		df_ua = df[df.file == "ParlaMint-UA_2013-06-18-m1-uk.conllu"][:30]

		print("Sample files has {} sentences and {} words.".format(df_ua["text"].count(), df_ua["length"].sum()))

		# Create a list of sentences from the df
		sentence_list = df_ua.text.to_list()

		# Translate the sample using all available models for this language
		for opus_lang_code in lang_models_dict[lang_code]:
			translation_list = model.translate(sentence_list, source_lang = "{}".format(opus_lang_code), target_lang='en')

			# Add the translations to the df
			df_ua["translation-{}".format(opus_lang_code)] = translation_list
		
		df_ua = df_ua.drop(columns=["file", "sentence_id", "tokenized_text", "proper_nouns", "length"])

		# Save the df
		df_ua.to_csv("/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-uk-sample-model-comparison.csv".format(lang_code, lang_code))

		print("The file is saved as/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-uk-sample-model-comparison.csv. ".format(lang_code, lang_code))

		# Create also a Russian sample
		# Create a sample, making sure that it is in Russian language and that the file is big enough (has 30 sentences)
		df_ru = df[df.file == 'ParlaMint-UA_2013-05-24-m0-ru.conllu'][:30]

		print("Russian sample files has {} sentences and {} words.".format(df_ru["text"].count(), df_ru["length"].sum()))

		# Create a list of sentences from the df
		ru_sentence_list = df_ru.text.to_list()

		# Translate the sample using all available models for this language
		for opus_lang_code in lang_models_dict["UA_ru"]:
			ru_translation_list = model.translate(ru_sentence_list, source_lang = "{}".format(opus_lang_code), target_lang='en')

			# Add the translations to the df
			df_ru["translation-{}".format(opus_lang_code)] = ru_translation_list
		
		df_ru = df_ru.drop(columns=["file", "sentence_id", "tokenized_text", "proper_nouns", "length"])

		# Save the df
		df_ru.to_csv("/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-ru-sample-model-comparison.csv".format(lang_code, lang_code))

		print("The file is saved as/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-ru-sample-model-comparison.csv. ".format(lang_code, lang_code))

	elif lang_code == "ES-CT":
			# Create a sample, making sure that it is in Catalan language and that the file is big enough (has 30 sentences)
		df_ua = df[df.file == "ParlaMint-ES-CT_2017-07-13-3902-ca.conllu"][:30]

		print("Sample files has {} sentences and {} words.".format(df_ua["text"].count(), df_ua["length"].sum()))

		# Create a list of sentences from the df
		sentence_list = df_ua.text.to_list()

		# Translate the sample using all available models for this language
		for opus_lang_code in lang_models_dict[lang_code]:
			translation_list = model.translate(sentence_list, source_lang = "{}".format(opus_lang_code), target_lang='en')

			# Add the translations to the df
			df_ua["translation-{}".format(opus_lang_code)] = translation_list
		
		df_ua = df_ua.drop(columns=["file", "sentence_id", "tokenized_text", "proper_nouns", "length"])

		# Save the df
		df_ua.to_csv("/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-ct-sample-model-comparison.csv".format(lang_code, lang_code))

		print("The file is saved as/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-ct-sample-model-comparison.csv. ".format(lang_code, lang_code))

		# Create also a Spanish sample
		# Create a sample, making sure that it is in Spanish language and that the file is big enough (has 30 sentences)
		df_ru = df[df.file == 'ParlaMint-ES-CT_2017-10-10-4301-es.conllu'][:30]

		print("Spanish sample files has {} sentences and {} words.".format(df_ru["text"].count(), df_ru["length"].sum()))

		# Create a list of sentences from the df
		ru_sentence_list = df_ru.text.to_list()

		# Translate the sample using all available models for this language
		for opus_lang_code in lang_models_dict["ES"]:
			ru_translation_list = model.translate(ru_sentence_list, source_lang = "{}".format(opus_lang_code), target_lang='en')

			# Add the translations to the df
			df_ru["translation-{}".format(opus_lang_code)] = ru_translation_list
		
		df_ru = df_ru.drop(columns=["file", "sentence_id", "tokenized_text", "proper_nouns", "length"])

		# Save the df
		df_ru.to_csv("/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-es-sample-model-comparison.csv".format(lang_code, lang_code))

		print("The file is saved as/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-es-sample-model-comparison.csv. ".format(lang_code, lang_code))
	else:
		# Create a smaller sample - just a couple of sentences from one file
		df = df[df.file == list(df["file"].unique())[0]][:30]

		print("Sample files has {} sentences and {} words.".format(df["text"].count(), df["length"].sum()))

		# Create a list of sentences from the df
		sentence_list = df.text.to_list()

		# Translate the sample using all available models for this language
		for opus_lang_code in lang_models_dict[lang_code]:
			translation_list = model.translate(sentence_list, source_lang = "{}".format(opus_lang_code), target_lang='en')

			# Add the translations to the df
			df["translation-{}".format(opus_lang_code)] = translation_list
		
		df = df.drop(columns=["file", "sentence_id", "tokenized_text", "proper_nouns", "length"])

		# For Portuguese, let's try another Portuguese model that is on HF, but is not OPUS-MT
		if lang_code == "PT":
			new_translation_list = []

			from transformers import MBart50TokenizerFast, MBartForConditionalGeneration
			ckpt = 'Narrativa/mbart-large-50-finetuned-opus-pt-en-translation'

			tokenizer = MBart50TokenizerFast.from_pretrained(ckpt)
			model = MBartForConditionalGeneration.from_pretrained(ckpt).to("cuda")

			tokenizer.src_lang = 'pt_XX'

			def translate(text):
				inputs = tokenizer(text, return_tensors='pt')
				input_ids = inputs.input_ids.to('cuda')
				attention_mask = inputs.attention_mask.to('cuda')
				output = model.generate(input_ids, attention_mask=attention_mask, forced_bos_token_id=tokenizer.lang_code_to_id['en_XX'])
				return tokenizer.decode(output[0], skip_special_tokens=True)

			for sentence in sentence_list:
				translation = translate(sentence)
				new_translation_list.append(translation)
			
			df["translation-narrativa"] = new_translation_list
		
		# For Greek, let's try another model, that does not work with EasyNMT, but is on HF
		if lang_code == "GR":
			from transformers import pipeline
			new_translation_list = []
			pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-el-en")
			for sentence in sentence_list:
				translation = pipe(sentence)
				new_translation_list.append(translation[0]["translation_text"])
			df["translation-tc-big-el-en"] = new_translation_list

		# Save the df
		df.to_csv("/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-sample-model-comparison.csv".format(lang_code, lang_code))

		print("The file is saved as/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-sample-model-comparison.csv. ".format(lang_code, lang_code))

	return df


In [5]:
df = choose_model(lang_code, extracted_dataframe_path)

Entire corpus has 928303 sentences and 13527989 words.
Sample files has 30 sentences and 521 words.


/home/tajak/Parlamint-translation/parlamint_env/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/tajak/Parlamint-translation/parlamint_env/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/tajak/Parlamint-translation/parl

The file is saved as/home/tajak/Parlamint-translation/results/FI/ParlaMint-FI-sample-model-comparison.csv. 


In [10]:
# Then open the sample and manually evaluate which model is better in the column "comparison"
# Open the analysed sample

sample = pd.read_csv("/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-sample-model-comparison.csv".format(lang_code, lang_code), index_col = 0)
sample.head(2)

,file_path,text,translation-et,translation-urj,translation-fiu
0,/home/tajak/Parlamint-translation/Source-data/...,"Austatud Riigikogu, tere hommikust!","Dear Riigikogu, good morning!","Mr. Riigikogu, good morning.","Good morning, Riigikogu."
1,/home/tajak/Parlamint-translation/Source-data/...,Alustame Riigikogu täiskogu VI istungjärgu kuu...,We will begin the session of the sixth working...,We are starting the Tuesday session of the 6th...,We will begin the 6th working week of the plen...


In [6]:
sample.comparison.value_counts()

zls    10
sla     1
Name: comparison, dtype: int64

We will use the "zls" model.